In [1]:
from helpers import backends, detector, models, df_metrics, delete_representations, get_id, visulaize_frame
import cv2
from cv2 import VideoCapture
from deepface import DeepFace
import os

C:\Users\biodu\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
def capture_image_from_cam(id):
    cap = cv2.VideoCapture(0)
    
    while True:
        ret, frame = cap.read()  
        face = detector(frame)

        if face:
            x, y, w, h = face[0]['facial_area'].values()
            
            # Draw rectangle around face  
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 2)
            
            # Crop image
            crop_img = frame[y:y+h, x:x+w]
            
            # Display webcam preview
            cv2.imshow('Register Face', frame)
    
            # Check for capture command  
            if cv2.waitKey(1) & 0xFF == ord('c'):
                return crop_img
            
        else:
            # Display helper text
            h, w = frame.shape[:2]
            cv2.putText(frame, "No face detected", 
                       (10, h - 20), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)
            
            # Display webcam preview            
            cv2.imshow('Register Face', frame)
                      
        # Check for exit    
        if cv2.waitKey(1) & 0xFF == ord('q'):
            return None
                      
    cap.release()

In [3]:
def register_face(id, image_path=None, live = False):
    result = {"success": False, "message": "", "id": id, "image_path": ""}

    # check for duplicate id
    if id + '.jpg' in os.listdir(os.getcwd() + '/img_db/'):
        print("id already exists")
        result["message"] = "id already exists"
        check = input("Enter new id or enter 'x' to quit: ").lower()
        if check == 'x':
            print("exiting command...")
            result["message"] = "exiting command..."
            return result
        else:
            id = check
            
    if live:
        live_capture = capture_image_from_cam(id)
        if live_capture is not None:
            # Save face
            face_path = os.path.join(os.getcwd() + '/img_db/', id + '.jpg')
            # face_path = "/img_db/"+id+".jpg"
            cv2.imwrite(face_path, live_capture)
            print(f" id {id} registered successfully")
            print(f"image stored in {face_path}")

            delete_representations()
            result["success"] = True
            result["message"] = f"id {id} registered successfully"
            result["image_path"] = face_path
            return result

    else:
        # check image_path is not None
        if image_path is None:
            result["message"] = "image_path is None"
            print(result["message"])
            return result
        
        frame = cv2.imread(image_path)

        # perform face detection
        face = detector(frame)
        if face:
            # crop and save image
            x, y , w, h = face[0]['facial_area'].values()
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 3)
            crop_img = frame[y:y+h, x:x+w]

            # Display the image with the detected face
            cv2.imshow('detect face', frame)
            # cv2.waitKey(0)

            # Save face
            face_path = os.path.join(os.getcwd() + '/img_db/', id + '.jpg')
            # face_path = "/img_db/"+id+".jpg"
            cv2.imwrite(face_path, crop_img)
            print(f" id {id} registered successfully")
            print(f"image stored in {face_path}")

            delete_representations()
            result["success"] = True
            result["message"] = f"id {id} registered successfully"
            result["image_path"] = face_path
            return result
        
        else:
            result["message"] = "No face detected in the provided image"
            print(result["message"])
            cv2.destroyAllWindows()
            return result

In [6]:
img = capture_image_from_cam('mme173416')